**Question 1**: Download relevant data

- nycflights13_flights
- nycflights13_airlines
- nycflights13_airports
- nycflights13_planes
- nycflights13_weather

**Question 1**: Import and set up SQLite

In [ ]:
# Import packages
import sqlite3

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Read the files
# Flight, Airline, Airport, Plane and Weather data
flight= pd.read_csv("nycflights13_flights_cleaned.csv")
airline= pd.read_csv("nycflights13_airlines_cleaned.csv")
airport= pd.read_csv("nycflights13_airports_cleaned.csv")
plane= pd.read_csv("nycflights13_planes_cleaned.csv")
weather= pd.read_csv("nycflights13_weather_cleaned.csv")

In [ ]:
import tempfile, os.path
dbfile = os.path.join(tempfile.mkdtemp(), "flight.db")
print(dbfile)

/tmp/tmpy1q4c2rv/flight.db


In [ ]:
conn = sqlite3.connect(dbfile)

In [ ]:
airline.to_sql("airline", conn, index= False)
airport.to_sql("airport", conn, index= False)
flight.to_sql("flight", conn, index= False)
plane.to_sql("plane", conn, index= False)
weather.to_sql("weather", conn, index= False)

Comment: All the steps above help us in setting up the dtaabase connection according to the instruction shown in Module 4.6

**Query 1.1**: Select distinct engine from planes

In [ ]:
# SQL code
task1_sql = pd.read_sql_query("""
SELECT DISTINCT ENGINE FROM PLANE
""", conn)

task1_sql.head(5)

,engine
0,Turbo-fan
1,Turbo-jet
2,Reciprocating
3,4 Cycle
4,Turbo-shaft


In [ ]:
# Pandas code
task1_my= (pd.DataFrame(data= plane["engine"].unique(), columns= ["engine"]))

task1_my.head(5)

,engine
0,Turbo-fan
1,Turbo-jet
2,Reciprocating
3,4 Cycle
4,Turbo-shaft


In [ ]:
pd.testing.assert_frame_equal(task1_sql, task1_my)

Comment: To reach the similar output from the SQL command, unique() function has been apply to column engine in plane dataset to extract unique value.

**Query 1.2**: Select distinct type, engine from planes

In [ ]:
# SQL code
task2_sql= pd.read_sql_query("""
SELECT DISTINCT TYPE, ENGINE FROM PLANE
""", conn)

task2_sql.head(5)

,type,engine
0,Fixed wing multi engine,Turbo-fan
1,Fixed wing multi engine,Turbo-jet
2,Fixed wing single engine,Reciprocating
3,Fixed wing multi engine,Reciprocating
4,Fixed wing single engine,4 Cycle


In [ ]:
# Pandas code
task2_my= (pd.DataFrame(data= plane[["type", "engine"]].drop_duplicates().reset_index(drop= True)))

task2_my.head(5)

,type,engine
0,Fixed wing multi engine,Turbo-fan
1,Fixed wing multi engine,Turbo-jet
2,Fixed wing single engine,Reciprocating
3,Fixed wing multi engine,Reciprocating
4,Fixed wing single engine,4 Cycle


In [ ]:
pd.testing.assert_frame_equal(task2_sql, task2_my)

Comment: Instead of applying unique() function (the most common way for one column), drop_duplicate() function has been applied to filter two columns to reach the output of SQL.

**Query 1.3**: Select count (*), engine from planes group by engine

In [ ]:
# Given that value_count does not work sometime, we implement .size() instead
# SQL code
task3_sql= pd.read_sql_query("""
SELECT COUNT(*), ENGINE FROM PLANE GROUP BY ENGINE
""", conn)

task3_sql.head(5)

,COUNT(*),engine
0,2,4 Cycle
1,28,Reciprocating
2,2750,Turbo-fan
3,535,Turbo-jet
4,2,Turbo-prop


In [ ]:
# Pandas code
task3_my= plane.groupby(["engine"]).size().reset_index(name= "COUNT(*)")
task3_my= task3_my[["COUNT(*)", "engine"]]

task3_my.head(5)

,COUNT(*),engine
0,2,4 Cycle
1,28,Reciprocating
2,2750,Turbo-fan
3,535,Turbo-jet
4,2,Turbo-prop


In [ ]:
pd.testing.assert_frame_equal(task3_sql, task3_my)

Comment: Size() is being used to count the unique value based on the unique value of "engine".

**Query 1.4**: Select count (*), engine, type form planes group by engine, type

In [ ]:
# Given that value_count does not work sometime, we implement .size() instead
# SQL code
task4_sql= pd.read_sql_query("""
SELECT COUNT(*), ENGINE, TYPE FROM PLANE GROUP BY ENGINE, TYPE
""", conn)

task4_sql.head(5)

,COUNT(*),engine,type
0,2,4 Cycle,Fixed wing single engine
1,5,Reciprocating,Fixed wing multi engine
2,23,Reciprocating,Fixed wing single engine
3,2750,Turbo-fan,Fixed wing multi engine
4,535,Turbo-jet,Fixed wing multi engine


In [ ]:
# Pandas code
task4_my= plane.groupby(["engine", "type"]).size().reset_index(name= "COUNT(*)")
task4_my= task4_my[["COUNT(*)", "engine", "type"]]

task4_my.head(5)

,COUNT(*),engine,type
0,2,4 Cycle,Fixed wing single engine
1,5,Reciprocating,Fixed wing multi engine
2,23,Reciprocating,Fixed wing single engine
3,2750,Turbo-fan,Fixed wing multi engine
4,535,Turbo-jet,Fixed wing multi engine


In [ ]:
pd.testing.assert_frame_equal(task4_sql, task4_my)

Comment: size() function has been applied to count the occurence that is group by engine and type.

**Query 1.5**:
SELECT MIN(year), AVG(year), MAX(year), engine, manufacturer
FROM planes
GROUP BY engine, manufacturer

In [ ]:
task5_sql= pd.read_sql_query("""SELECT MIN(YEAR), AVG(YEAR), MAX(YEAR), ENGINE, MANUFACTURER FROM PLANE GROUP BY ENGINE, MANUFACTURER""", conn)
task5_sql.head(5)

In [ ]:
# Instead of apply min(), max() and mean() function,
task5_my= plane.groupby(["engine", "manufacturer"]).year.aggregate(["min", "mean", "max"]).reset_index()
task5_my= task5_my[["min", "mean", "max", "engine", "manufacturer"]]
task5_my= task5_my.rename(columns={"min": "MIN(YEAR)", "mean": "AVG(YEAR)", "max": "MAX(YEAR)"})

task5_my.head(5)

,MIN(YEAR),AVG(YEAR),MAX(YEAR),engine,manufacturer
0,1975.0,1975.0,1975.0,4 Cycle,CESSNA
1,NaN,NaN,NaN,4 Cycle,JOHN G HESS
2,NaN,NaN,NaN,Reciprocating,AMERICAN AIRCRAFT INC
3,2007.0,2007.0,2007.0,Reciprocating,AVIAT AIRCRAFT INC
4,NaN,NaN,NaN,Reciprocating,BARKER JACK L


In [ ]:
pd.testing.assert_frame_equal(task5_sql, task5_my)

Comment: After various methods attempted, I found that aggregate() function is the most efficient method to compute min, max and average (year) groupby engine and manufacturer.

**Query 1.6**:
SELECT * FROM planes WHERE speed IS NOT NULL

In [ ]:
# Given that value_count does not work sometime, we implement .size() instead
# SQL code
task6_sql= pd.read_sql_query("""
SELECT * FROM PLANE WHERE SPEED IS NOT NULL
""", conn)

task6_sql.head(5)

,tailnum,year,type,manufacturer,model,engines,seats,speed,engine
0,N201AA,1959.0,Fixed wing single engine,CESSNA,150,1,2,90.0,Reciprocating
1,N202AA,1980.0,Fixed wing multi engine,CESSNA,421C,2,8,90.0,Reciprocating
2,N350AA,1980.0,Fixed wing multi engine,PIPER,PA-31-350,2,8,162.0,Reciprocating
3,N364AA,1973.0,Fixed wing multi engine,CESSNA,310Q,2,6,167.0,Reciprocating
4,N378AA,1963.0,Fixed wing single engine,CESSNA,172E,1,4,105.0,Reciprocating


In [ ]:
# Pandas code
task6_my= plane.dropna().reset_index()
task6_my= task6_my[["tailnum", "year", "type", "manufacturer", "model", "engines", "seats", "speed", "engine"]]

task6_my.head(5)

,tailnum,year,type,manufacturer,model,engines,seats,speed,engine
0,N201AA,1959.0,Fixed wing single engine,CESSNA,150,1,2,90.0,Reciprocating
1,N202AA,1980.0,Fixed wing multi engine,CESSNA,421C,2,8,90.0,Reciprocating
2,N350AA,1980.0,Fixed wing multi engine,PIPER,PA-31-350,2,8,162.0,Reciprocating
3,N364AA,1973.0,Fixed wing multi engine,CESSNA,310Q,2,6,167.0,Reciprocating
4,N378AA,1963.0,Fixed wing single engine,CESSNA,172E,1,4,105.0,Reciprocating


In [ ]:
pd.testing.assert_frame_equal(task6_sql, task6_my)

Comment: In the case, dropna() has been used to diaplay the all Non-NA values within the dataframe.

**Query 1.7**:
SELECT tailnum FROM planes
WHERE seats BETWEEN 150 AND 190 AND year >= 2012


In [ ]:
# SQL code
task7_sql= pd.read_sql_query("""
SELECT TAILNUM FROM PLANE WHERE SEATS BETWEEN 150 AND 190 AND YEAR >= 2012
""", conn)

task7_sql.head(5)

,tailnum
0,N361VA
1,N849VA
2,N851VA
3,N852VA
4,N853VA


In [ ]:
# Pandas code
task7_my= plane.loc[(plane["seats"]>= 150) & (plane["seats"]<= 190) & (plane["year"]>= 2012)]
task7_my= task7_my[["tailnum"]].reset_index(drop= True)

task7_my.head(5)

,tailnum
0,N361VA
1,N849VA
2,N851VA
3,N852VA
4,N853VA


In [ ]:
pd.testing.assert_frame_equal(task7_sql, task7_my)

Comment: Loc function has been used with conditional filter stated to reach the same output display by the SQL command. If any case, index based method filter is need, iloc() function can be applied.

**Query 1.8**:
SELECT tailnum, manufacturer, seats FROM planes
WHERE manufacturer IN ("BOEING", "AIRBUS", "EMBRAER") AND seats>390

In [ ]:
# Given that value_count does not work sometime, we implement .size() instead
# SQL code
task8_sql= pd.read_sql_query("""
SELECT TAILNUM, MANUFACTURER, SEATS FROM PLANE WHERE MANUFACTURER IN ("BOEING", "AIRBUS", "EMBRAER") AND SEATS > 390
""", conn)

task8_sql.head(5)

,tailnum,manufacturer,seats
0,N206UA,BOEING,400
1,N228UA,BOEING,400
2,N272AT,BOEING,400
3,N57016,BOEING,400
4,N670US,BOEING,450


In [ ]:
# Pandas code
task8_my= plane.loc[(plane["seats"]>= 390)]
task8_my= task8_my[["tailnum", "manufacturer", "seats"]].reset_index(drop= True)

task8_my.head(5)

,tailnum,manufacturer,seats
0,N206UA,BOEING,400
1,N228UA,BOEING,400
2,N272AT,BOEING,400
3,N57016,BOEING,400
4,N670US,BOEING,450


In [ ]:
pd.testing.assert_frame_equal(task8_sql, task8_my)

Comment: Similar technique as demonstrated in Query 1.7 has been applied here to reach the output with different conditional filtered stated. From what we can see, the only manufacturer produce > 390 seats is BOEING.

**Query 1.9**:
SELECT DISTINCT year, seats FROM planes
WHERE year >= 2012 ORDER BY year ASC, seats DESC


In [ ]:
# SQL code
task9_sql= pd.read_sql_query("""
SELECT DISTINCT YEAR, SEATS FROM PLANE WHERE YEAR >= 2012 ORDER BY YEAR ASC, SEATS DESC
""", conn)

task9_sql.head(5)

,year,seats
0,2012.0,379
1,2012.0,377
2,2012.0,260
3,2012.0,222
4,2012.0,200


In [ ]:
# Pandas code
task9_my= plane.loc[(plane["year"]>= 2012), ["year", "seats"]].drop_duplicates()
task9_my= task9_my.sort_values(["year", "seats"], ascending= (True, False)).reset_index(drop= True)

task9_my.head(5)

,year,seats
0,2012.0,379
1,2012.0,377
2,2012.0,260
3,2012.0,222
4,2012.0,200


In [ ]:
pd.testing.assert_frame_equal(task9_sql, task9_my)

Comment: In the section, both conditional filter and unique value has been used. Nevertheless, sort_value() function has also been applied to sort the value ascendingly and descendingly. From what has been displayed, 379 seats plane is the largest plane while satisfying the condition of year >= 2012.

**Query 1.10**:
SELECT DISTINCT year, seats FROM planes
WHERE year >= 2012 ORDER BY seats DESC, year ASC

In [ ]:
# SQL code
task10_sql= pd.read_sql_query("""
SELECT DISTINCT YEAR, SEATS FROM PLANE WHERE YEAR >= 2012 ORDER BY SEATS DESC, YEAR ASC
""", conn)

task10_sql.head(5)

,year,seats
0,2012.0,379
1,2013.0,379
2,2012.0,377
3,2013.0,377
4,2012.0,260


In [ ]:
# Pandas code
task10_my= plane.loc[(plane["year"]>= 2012), ["year", "seats"]].drop_duplicates()
task10_my= task10_my.sort_values(["seats", "year"], ascending= (False, True)).reset_index(drop= True)

task10_my.head(5)

,year,seats
0,2012.0,379
1,2013.0,379
2,2012.0,377
3,2013.0,377
4,2012.0,260


In [ ]:
pd.testing.assert_frame_equal(task10_sql, task10_my)

Comment: Similar strategy applied in 1.9 has been applied here.

**Query 1.11**:
SELECT manufacturer, COUNT(*) FROM planes
WHERE seats > 200 GROUP BY manufacturer


In [ ]:
# SQL code
task11_sql= pd.read_sql_query("""
SELECT MANUFACTURER, COUNT(*) FROM PLANE WHERE SEATS> 200 GROUP BY MANUFACTURER
""", conn)

task11_sql.head(5)

,manufacturer,COUNT(*)
0,AIRBUS,66
1,AIRBUS INDUSTRIE,4
2,BOEING,225


In [ ]:
# Pandas code
task11_my= plane.loc[(plane["seats"]> 200)]
task11_my= pd.DataFrame(data= task11_my.groupby(["manufacturer"]).size().reset_index())
task11_my.rename(columns= {0: "COUNT(*)"}, inplace= True)

task11_my.head(5)

,manufacturer,COUNT(*)
0,AIRBUS,66
1,AIRBUS INDUSTRIE,4
2,BOEING,225


In [ ]:
pd.testing.assert_frame_equal(task11_sql, task11_my)

Comment: From what we observed, BOEING is the largest manfacturer that manufacture plane with > 200 seats.

**Query 1.12**:
SELECT manufacturer, COUNT(*) FROM planes
GROUP BY manufacturer HAVING COUNT(*) > 10

In [ ]:
# SQL code
task12_sql= pd.read_sql_query("""
SELECT MANUFACTURER, COUNT(*) FROM PLANE GROUP BY MANUFACTURER HAVING COUNT()> 10
""", conn)

task12_sql.head(5)

,manufacturer,COUNT(*)
0,AIRBUS,336
1,AIRBUS INDUSTRIE,400
2,BOEING,1630
3,BOMBARDIER INC,368
4,EMBRAER,299


In [ ]:
# Pandas code
task12_my= plane[["manufacturer"]]
task12_my= pd.DataFrame(data= task12_my.groupby(["manufacturer"]).size())
task12_my= task12_my.loc[(task12_my[0]>10)].reset_index()
task12_my.rename(columns= {0: "COUNT(*)"}, inplace= True)

task12_my.head(5)

,manufacturer,COUNT(*)
0,AIRBUS,336
1,AIRBUS INDUSTRIE,400
2,BOEING,1630
3,BOMBARDIER INC,368
4,EMBRAER,299


In [ ]:
pd.testing.assert_frame_equal(task12_sql, task12_my)

Comment: From the onservation, BOEING appear to be the biggest and dominate the market of plane manufacturer, following by AIRBUS Industries

**Query 1.13**:
SELECT manufacturer, COUNT(*) FROM planes
WHERE seats > 200 GROUP BY manufacturer HAVING COUNT(*) > 10

In [ ]:
# SQL code
task13_sql= pd.read_sql_query("""
SELECT MANUFACTURER, COUNT(*) FROM PLANE WHERE SEATS > 200 GROUP BY MANUFACTURER HAVING COUNT(*) > 10
""", conn)

task13_sql.head(5)

,manufacturer,COUNT(*)
0,AIRBUS,66
1,BOEING,225


In [ ]:
# Pandas code
task13_my= plane.loc[(plane["seats"]> 200)]
task13_my= pd.DataFrame(data= task13_my.groupby(["manufacturer"]).size())
task13_my=  task13_my.loc[(task13_my[0]>10)].reset_index()
task13_my.rename(columns= {0: "COUNT(*)"}, inplace= True)

task13_my.head(5)

,manufacturer,COUNT(*)
0,AIRBUS,66
1,BOEING,225


In [ ]:
pd.testing.assert_frame_equal(task13_sql, task13_my)

Comment: From what it can be observed, BOEING & AIRBUS are sharing the plane manufacturing market, amnufacturing plane more than 200 plane seats.

**Query 1.14**: SELECT manufacturer, COUNT(*) AS howmany FROM planes GROUP BY manufacturer ORDER BY howmany DESC LIMIT 5

In [ ]:
# SQL code
task14_sql= pd.read_sql_query("""
SELECT MANUFACTURER, COUNT(*) AS HOWMANY FROM PLANE GROUP BY MANUFACTURER ORDER BY HOWMANY DESC LIMIT 5
""", conn)

task14_sql.head(5)

,manufacturer,HOWMANY
0,BOEING,1630
1,AIRBUS INDUSTRIE,400
2,BOMBARDIER INC,368
3,AIRBUS,336
4,EMBRAER,299


In [ ]:
# Pandas code
task14_my= plane[["manufacturer"]]
task14_my= pd.DataFrame(data= task14_my.groupby(["manufacturer"]).size())
task14_my= task14_my.sort_values(0, ascending= False).head(5).reset_index()
task14_my.rename(columns= {0: "HOWMANY"}, inplace= True)

task14_my.head(5)

,manufacturer,HOWMANY
0,BOEING,1630
1,AIRBUS INDUSTRIE,400
2,BOMBARDIER INC,368
3,AIRBUS,336
4,EMBRAER,299


In [ ]:
pd.testing.assert_frame_equal(task14_sql, task14_my)

Comment: From what we can see grouping by manufacturer, BOEING is dominating the market.

**Question 1.15**: SELECT flights.*, planes.year AS plane_year, planes.speed AS plane_speed, planes.seats AS plane_seats FROM flights LEFT JOIN planes ON flights.tailnum=planes.tailnum

In [ ]:
# SQL code
task15_sql= pd.read_sql_query("""
                              SELECT FLIGHT.*,
                              PLANE.YEAR AS PLANE_YEAR,
                              PLANE.SPEED AS PLANE_SPEED,
                              PLANE.SEATS AS PLANE_SEATS
                              FROM FLIGHT
                              LEFT JOIN PLANE ON FLIGHT.TAILNUM= PLANE.TAILNUM
                              """, conn)

task15_sql.head(5)

,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,...,origin,dest,air_time,distance,hour,minute,time_hour,PLANE_YEAR,PLANE_SPEED,PLANE_SEATS
0,2013,1,1,517.0,515,2.0,830.0,819,11.0,UA,...,EWR,IAH,227.0,1400,5,15,1/01/2013 5:00,1999.0,NaN,149.0
1,2013,1,1,533.0,529,4.0,850.0,830,20.0,UA,...,LGA,IAH,227.0,1416,5,29,1/01/2013 5:00,1998.0,NaN,149.0
2,2013,1,1,542.0,540,2.0,923.0,850,33.0,AA,...,JFK,MIA,160.0,1089,5,40,1/01/2013 5:00,1990.0,NaN,178.0
3,2013,1,1,544.0,545,-1.0,1004.0,1022,-18.0,B6,...,JFK,BQN,183.0,1576,5,45,1/01/2013 5:00,2012.0,NaN,200.0
4,2013,1,1,554.0,600,-6.0,812.0,837,-25.0,DL,...,LGA,ATL,116.0,762,6,0,1/01/2013 6:00,1991.0,NaN,178.0


In [ ]:
# Pandas code
df= plane[["year", "speed", "seats", "tailnum"]]
task15_my= flight.merge(df, how= "left", on= "tailnum")
task15_my.rename(columns= {"year_x": "year", "year_y": "PLANE_YEAR", "speed": "PLANE_SPEED", "seats": "PLANE_SEATS"}, inplace= True)

task15_my.head(5)

,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,...,origin,dest,air_time,distance,hour,minute,time_hour,PLANE_YEAR,PLANE_SPEED,PLANE_SEATS
0,2013,1,1,517.0,515,2.0,830.0,819,11.0,UA,...,EWR,IAH,227.0,1400,5,15,1/01/2013 5:00,1999.0,NaN,149.0
1,2013,1,1,533.0,529,4.0,850.0,830,20.0,UA,...,LGA,IAH,227.0,1416,5,29,1/01/2013 5:00,1998.0,NaN,149.0
2,2013,1,1,542.0,540,2.0,923.0,850,33.0,AA,...,JFK,MIA,160.0,1089,5,40,1/01/2013 5:00,1990.0,NaN,178.0
3,2013,1,1,544.0,545,-1.0,1004.0,1022,-18.0,B6,...,JFK,BQN,183.0,1576,5,45,1/01/2013 5:00,2012.0,NaN,200.0
4,2013,1,1,554.0,600,-6.0,812.0,837,-25.0,DL,...,LGA,ATL,116.0,762,6,0,1/01/2013 6:00,1991.0,NaN,178.0


In [ ]:
pd.testing.assert_frame_equal(task15_sql, task15_my)

Comment: We first define df with columns- year, speed, seats and tailnum from plane. Then, we merge both flight dataframe and df dataframe with left join on tailnum, following by renaming the column name.

**Question 1.16**: SELECT planes.*, airlines.* FROM (SELECT DISTINCT carrier, tailnum FROM flights) AS cartail INNER JOIN planes ON cartail.tailnum=planes.tailnum
INNER JOIN airlines ON cartail.carrier= airlines.carrier

In [ ]:
# SQL code
task16_sql= pd.read_sql_query("""
                              SELECT PLANE.*,
                              AIRLINE. * FROM (SELECT DISTINCT CARRIER, TAILNUM FROM FLIGHT) AS CARTAIL
                              INNER JOIN PLANE ON CARTAIL.TAILNUM= PLANE.TAILNUM
                              INNER JOIN AIRLINE ON CARTAIL.CARRIER= AIRLINE.CARRIER
                              """, conn)
task16_sql.head(5)

,tailnum,year,type,manufacturer,model,engines,seats,speed,engine,carrier,name
0,N10156,2004.0,Fixed wing multi engine,EMBRAER,EMB-145XR,2,55,NaN,Turbo-fan,EV,ExpressJet Airlines Inc.
1,N102UW,1998.0,Fixed wing multi engine,AIRBUS INDUSTRIE,A320-214,2,182,NaN,Turbo-fan,US,US Airways Inc.
2,N103US,1999.0,Fixed wing multi engine,AIRBUS INDUSTRIE,A320-214,2,182,NaN,Turbo-fan,US,US Airways Inc.
3,N104UW,1999.0,Fixed wing multi engine,AIRBUS INDUSTRIE,A320-214,2,182,NaN,Turbo-fan,US,US Airways Inc.
4,N10575,2002.0,Fixed wing multi engine,EMBRAER,EMB-145LR,2,55,NaN,Turbo-fan,EV,ExpressJet Airlines Inc.


In [ ]:
# Pandas code
CARTAIL= (pd.DataFrame(data= flight[["carrier", "tailnum"]].drop_duplicates().reset_index(drop= True)))
task16_my= airline.merge(CARTAIL, how= "inner", on= "carrier")
task16_my= plane.merge(task16_my, how= "inner", on= "tailnum")

task16_my.head(5)

,tailnum,year,type,manufacturer,model,engines,seats,speed,engine,carrier,name
0,N10156,2004.0,Fixed wing multi engine,EMBRAER,EMB-145XR,2,55,NaN,Turbo-fan,EV,ExpressJet Airlines Inc.
1,N102UW,1998.0,Fixed wing multi engine,AIRBUS INDUSTRIE,A320-214,2,182,NaN,Turbo-fan,US,US Airways Inc.
2,N103US,1999.0,Fixed wing multi engine,AIRBUS INDUSTRIE,A320-214,2,182,NaN,Turbo-fan,US,US Airways Inc.
3,N104UW,1999.0,Fixed wing multi engine,AIRBUS INDUSTRIE,A320-214,2,182,NaN,Turbo-fan,US,US Airways Inc.
4,N10575,2002.0,Fixed wing multi engine,EMBRAER,EMB-145LR,2,55,NaN,Turbo-fan,EV,ExpressJet Airlines Inc.


In [ ]:
pd.testing.assert_frame_equal(task16_sql, task16_my)

Comment: We first define a dataframe consisting carrier and tailnum named CARTAIL. Then, merging both CARTAIL and airline with inner join on carrier, later merging the dataframe with plane iwth inner join on tailnum to generate the smae output shown by SQL command.

**Task 2.17**: Implement the following codes

In [ ]:
# SQL code
task17_sql= pd.read_sql_query("""
                              SELECT
                              flights2.*,
                              atemp,
                              ahumid

                              FROM (
                              SELECT * FROM flight WHERE origin='EWR'
                              ) AS flights2

                              LEFT JOIN (

                              SELECT
                              year, month, day,
                              AVG(temp) AS atemp,
                              AVG(humid) AS ahumid
                              FROM weather
                              WHERE origin='EWR'

                              GROUP BY year, month, day
                              ) AS weather2

                              ON flights2.year=weather2.year
                              AND flights2.month=weather2.month
                              AND flights2.day=weather2.day""", conn)

task17_sql.head(5)

,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,...,tailnum,origin,dest,air_time,distance,hour,minute,time_hour,atemp,ahumid
0,2013,1,1,517.0,515,2.0,830.0,819,11.0,UA,...,N14228,EWR,IAH,227.0,1400,5,15,1/01/2013 5:00,38.48,58.386087
1,2013,1,1,554.0,558,-4.0,740.0,728,12.0,UA,...,N39463,EWR,ORD,150.0,719,5,58,1/01/2013 5:00,38.48,58.386087
2,2013,1,1,555.0,600,-5.0,913.0,854,19.0,B6,...,N516JB,EWR,FLL,158.0,1065,6,0,1/01/2013 6:00,38.48,58.386087
3,2013,1,1,558.0,600,-2.0,923.0,937,-14.0,UA,...,N53441,EWR,SFO,361.0,2565,6,0,1/01/2013 6:00,38.48,58.386087
4,2013,1,1,559.0,600,-1.0,854.0,902,-8.0,UA,...,N76515,EWR,LAS,337.0,2227,6,0,1/01/2013 6:00,38.48,58.386087


In [ ]:
# Pandas code
# Extract the core data
flight2= flight[(flight.origin== "EWR")]
weather2= weather[(weather.origin== "EWR")]

weather2= weather2.groupby(["year", "month", "day"])["temp", "humid"].mean().reset_index()
weather2= weather2.rename(columns= {"temp": "atemp", "humid": "ahumid"})

task17_my= pd.merge(flight2, weather2[["year", "month", "day", "atemp", "ahumid"]], how= "left")

task17_my.head(5)

<ipython-input-54-a58b598f794a>:6: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  weather2= weather2.groupby(["year", "month", "day"])["temp", "humid"].mean().reset_index()


,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,...,tailnum,origin,dest,air_time,distance,hour,minute,time_hour,atemp,ahumid
0,2013,1,1,517.0,515,2.0,830.0,819,11.0,UA,...,N14228,EWR,IAH,227.0,1400,5,15,1/01/2013 5:00,38.48,58.386087
1,2013,1,1,554.0,558,-4.0,740.0,728,12.0,UA,...,N39463,EWR,ORD,150.0,719,5,58,1/01/2013 5:00,38.48,58.386087
2,2013,1,1,555.0,600,-5.0,913.0,854,19.0,B6,...,N516JB,EWR,FLL,158.0,1065,6,0,1/01/2013 6:00,38.48,58.386087
3,2013,1,1,558.0,600,-2.0,923.0,937,-14.0,UA,...,N53441,EWR,SFO,361.0,2565,6,0,1/01/2013 6:00,38.48,58.386087
4,2013,1,1,559.0,600,-1.0,854.0,902,-8.0,UA,...,N76515,EWR,LAS,337.0,2227,6,0,1/01/2013 6:00,38.48,58.386087


In [ ]:
pd.testing.assert_frame_equal(task17_sql, task17_my)

Comment: To generate the same output, we first identify the sub-query (flight2 and weather2). With the sub-query rules out, we then group the weathers based on year, month, day while calculating the mean. Later, merge the dataframe with flight2 with left join to genertae the sam eoutput.

In [ ]:
# Package to convert to html
!pip install nbconvert

In [ ]:
!jupyter nbconvert --to html 5xDataframe.ipynb

In [ ]:
# Pacakge to convert to pdf
!sudo apt-get install pandoc
!apt-get install texlive texlive-xetex texlive-latex-extra pandoc

In [ ]:
!jupyter nbconvert --to pdf 5xDataframe.ipynb